<a href="https://colab.research.google.com/github/AI-zen-admin/manage-pinecone-index/blob/main/manage_pinecone_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 初期設定でやること

1. 「環境変数とPineconeの定義」で環境変数の設定
1. 「IDの読み込み」でids_pathとnamespaceの設定

# データアップロード
1. 「データ登録」より前のセルを全て実行
1. 「データ登録」でURLを設定して実行
1. 「データ登録」実行中にファイルアップロード
1. 「データ追加」を実行



In [ ]:
#@title パッケージインストール（編集不要）
%%capture captured
packages = """
langchain==0.0.232
openai
tiktoken
urllib3<2
pinecone-client

unstructured
selenium
pypdf
pypdfium2
docx2txt
"""

with open(f"./requirements.txt", "w") as f:
    f.write(packages)

!pip install -r requirements.txt

In [ ]:
#@title Webドライバーの設定（編集不要）
%%capture captured
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

In [ ]:
#@title 環境変数とPineconeの定義（初期設定 1）
import os
import pinecone

os.environ["INDEX_NAME"] = "INDEX_NAME"
os.environ["PINECONE_API_KEY"] = "PINECONE_API_KEY"
os.environ["PINECONE_ENV"] = "PINECONE_ENV"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
os.environ["NOTION_TOKEN"] = "NOTION_TOKEN"

# initialize pinecone
pinecone.init(
    api_key=os.environ["PINECONE_API_KEY"],  # find at app.pinecone.io
    environment=os.environ["PINECONE_ENV"]  # next to api key in console
)

In [ ]:
#@title IDの読み込み（初期設定 2）
import json

from google.colab import drive
drive.mount('/content/drive')

# IDを保存するファイルのパスを設定
ids_path = "/content/drive/MyDrive/Pinecone"

# 名前空間（インデックスの分類）を設定
namespace = "" # デフォルトは空

# カスタムメタデータを設定
additional_metadata = {} # デフォルトは空

try:
    with open(f"{ids_path}/ids_dict_{namespace}.json", "r") as f:
        current_ids_dict = json.load(f)
except FileNotFoundError:
    # IDを保存するファイルを作成
    with open(f"{ids_path}/ids_dict_{namespace}.json", "w") as f:
        f.write("{}")
    current_ids_dict = {}

!mkdir load_files
load_dir = "/content/load_files"

In [ ]:
#@title カスタムクラス（編集不要）
from typing import Any, Dict, List, Optional
from langchain.docstore.document import Document

from langchain.document_loaders import NotionDBLoader

import pickle

NOTION_BASE_URL = "https://api.notion.com/v1"
DATABASE_URL = NOTION_BASE_URL + "/databases/{database_id}/query"

class CustomNotionDBLoader(NotionDBLoader):
    def load(self) -> List[Document]:
        """Load documents from the Notion database.
        Returns:
            List[Document]: List of documents.
        """
        page_summaries = self._retrieve_page_summaries()

        try:
            with open(f'{ids_path}/previous_page_summaries.pkl', 'rb') as f:
                previous_page_summaries = pickle.load(f)
        except FileNotFoundError:
            previous_page_summaries = []

        previous_page_summaries_dict = {item['id']: item['last_edited_time'] for item in previous_page_summaries}

        updated_page_summaries = [page_summary for page_summary in page_summaries if
                                  page_summary['id'] not in previous_page_summaries_dict or
                                  page_summary['last_edited_time'] != previous_page_summaries_dict[page_summary['id']]]

        with open(f'{ids_path}/previous_page_summaries.pkl', 'wb') as f:
            pickle.dump(page_summaries, f)

        return list(self.load_page(page_summary) for page_summary in updated_page_summaries)

    def _retrieve_page_summaries(
        self, query_dict: Dict[str, Any] = {"page_size": 100}
    ) -> List[Dict[str, Any]]:
        """Get all the pages from a Notion database."""
        pages: List[Dict[str, Any]] = []

        while True:
            data = self._request(
                DATABASE_URL.format(database_id=self.database_id),
                method="POST",
                query_dict=query_dict,
            )

            pages.extend(data.get("results"))

            if not data.get("has_more"):
                break

            query_dict["start_cursor"] = data.get("next_cursor")

        return pages


In [16]:
#@title ManageIndexクラス（編集不要）
import os
import pinecone
import traceback
import uuid
import pandas as pd

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import (
    SeleniumURLLoader,
    PyPDFLoader,
    PDFMinerLoader,
    PyPDFium2Loader,
    CSVLoader,
    UnstructuredFileLoader,
    Docx2txtLoader,
    UnstructuredExcelLoader,
    UnstructuredPowerPointLoader,

)

from langchain.text_splitter import (
    MarkdownTextSplitter,
    CharacterTextSplitter
)


extension_dict = {
    '.txt': 'text',
    '.csv': 'csv',
    '.json': 'json',
    '.pdf': 'pdf',
    '.md': 'markdown',
    '.docx': 'word',
    'xlsx': 'excel',
    'pptx': 'power_point',

}
special_dirs_dict = {
    "NotionDB": "notion",
}


class ManageIndex:
    def __init__(self, load_dir, urls=None, specify_loader=[]):
        self.embeddings = OpenAIEmbeddings()
        self.index_name = os.environ["INDEX_NAME"]
        self.docs = []
        self.load_dir = load_dir
        self.urls = urls
        self.specify_loader = specify_loader

    def _load_pdf(self, path):
        try:
            if "PDFMinerLoader" in self.specify_loader:
                print(f"Try PDFMinerLoader: {path}")
                loader = PDFMinerLoader(path)
            else:
                print(f"Try PyPDFLoader: {path}")
                loader = PyPDFLoader(path)
            documents = loader.load()
        except Exception as e:
            print(traceback.format_exc())
            print(f"Try PyPDFium2Loader: {path}")
            loader = PyPDFium2Loader(path)
            documents = loader.load()

        text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)
        split_docs = text_splitter.split_documents(documents)
        return split_docs

    def _load_url(self, urls):
        loader = SeleniumURLLoader(urls)
        documents = loader.load()
        text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)
        split_docs = text_splitter.split_documents(documents)
        return split_docs

    def _load_csv(self, path):
        print("try CSVLoader")
        loader = CSVLoader(path)
        documents = loader.load()
        return documents

    def _load_text(self, path):
        print("try UnstructuredFileLoader")
        loader = UnstructuredFileLoader(path)
        documents = loader.load()
        text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)
        split_docs = text_splitter.split_documents(documents)
        return split_docs

    def _load_word(self, path):
        print("try Docx2txtLoader")
        loader = Docx2txtLoader(path)
        documents = loader.load()
        text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)
        split_docs = text_splitter.split_documents(documents)
        return split_docs

    def _load_excel(self, path):
        print("try UnstructuredExcelLoader")
        loader = UnstructuredExcelLoader(path)
        documents = loader.load()
        text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)
        split_docs = text_splitter.split_documents(documents)
        return split_docs

    def _load_power_point(self, path):
        print("try UnstructuredPowerPointLoader")
        loader = UnstructuredPowerPointLoader(path)
        documents = loader.load()
        text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)
        split_docs = text_splitter.split_documents(documents)
        return split_docs


    def load_notiondb(self, id):
        print("try CustomNotionDBLoader")
        loader = CustomNotionDBLoader(
            integration_token=os.environ["NOTION_TOKEN"],
            database_id=id,
            request_timeout_sec=30,  # optional, defaults to 10
        )
        documents = loader.load()
        text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)
        split_docs = text_splitter.split_documents(documents)
        return split_docs

    def _entry_documents(self, doc_type, doc_value):
        self.docs.append(
            {"type": doc_type, "value": doc_value}
        )

    def load_documents(self):
        # entry website
        urls = list(self.urls)
        if urls:
            # check notion url
            for url in urls[:]:
                if 'notion.so' in url:
                    # 'notion.so'以降の部分を抽出
                    url_part = url.split('notion.so')[1]
                    # パスの最初の部分を取得 (IDと想定)
                    path_parts = url_part.split('/')
                    if len(path_parts) > 2:
                        notion_id = path_parts[2].split('?')[0]
                        self._entry_documents("notion", notion_id)
                        urls.remove(url)  # urlをリストから削除
                    else:
                        print("Notion IDがURLから見つかりません。")

            self._entry_documents("url", urls)

        # entry files
        for root, _, files in os.walk(self.load_dir):
            dir_name = os.path.basename(root)
            if dir_name in special_dirs_dict:
                self._entry_documents(special_dirs_dict[dir_name], root)
                continue
            for file in files:
                file_path = os.path.join(root, file)
                _, ext = os.path.splitext(file_path)
                file_type = extension_dict.get(ext, ext)

                self._entry_documents(file_type, file_path)

        return self.docs

    def from_documents(self, namespace=None, additional_metadata={}):
        documents = []
        source_list = []
        for doc in self.docs:
            try:
                document = None
                if doc["type"] == "pdf":
                    document = self._load_pdf(doc["value"])
                elif doc["type"] == "markdown":
                    document = self._load_pdf(doc["value"])
                elif doc["type"] == "csv":
                    document = self._load_csv(doc["value"])
                elif doc["type"] == "url":
                    document = self._load_url(doc["value"])
                elif doc["type"] == "text":
                    document = self._load_text(doc["value"])
                elif doc["type"] == "word":
                    document = self._load_word(doc["value"])
                elif doc["type"] == "excel":
                    document = self._load_excel(doc["value"])
                elif doc["type"] == "power_point":
                    document = self._load_power_point(doc["value"])
                elif doc["type"] == "notion":
                    document = self._load_notiondb(doc["value"])

                if document:
                    documents.extend(document)
                    source_list.append(doc["value"])
            except Exception as e:
                print(traceback.format_exc())
                return doc["value"]

        ids_dict = {}
        if documents:
            # fix metadata
            for document in documents:
                document.metadata.update(**additional_metadata)

                source = document.metadata.get("source")
                if source:
                    document.metadata["source"] = source.replace(self.load_dir+"/", "")

                # metadataにidがあればそれをidとする
                id = document.metadata.get("id") or str(uuid.uuid4())
                ids_dict[id] = document.metadata
            print(ids_dict)
            print(documents)

            self.pinecone = Pinecone.from_documents(
                documents=documents,
                embedding=self.embeddings,
                ids=list(ids_dict.keys()),
                index_name=self.index_name,
                namespace=namespace,
            )
            print(ids_dict)
        return ids_dict

    def from_items(self, namespace=None):
        try:
            df = pd.read_csv(self.load_dir+"/source_documents.csv")

            ids_dict = {}
            texts = []
            metadatas = []
            ids = []
            for i in range(len(df)):
                row = df.iloc[i]

                # idとpage_content以外をmetadataとして保存
                metadata = {}
                metadata_include_content = {}
                for column in df.columns:
                    if column != "id" and pd.notna(row[column]):
                        metadata_include_content[column] = row[column]
                        if column != "page_content":
                            metadata[column] = row[column]

                ids_dict[row["id"]] = metadata_include_content
                texts.append(row["page_content"])
                metadatas.append(metadata)
                ids.append(row["id"])
        except Exception as e:
            print(traceback.format_exc())
            return f"Invalid file content: {e}"

        self.pinecone = Pinecone.from_texts(
            texts=texts,
            embedding=self.embeddings,
            metadatas=metadatas,
            ids=ids,
            index_name=self.index_name,
            namespace=namespace,
        )
        return ids_dict


In [36]:
#@title インデックス操作関数定義（編集不要）
def update_ids_json(upsert_ids_dict={}, delete_ids_list=[]):
    for id, metadata in upsert_ids_dict.items():
        current_ids_dict[id] = metadata

    for id in delete_ids_list:
        try:
            current_ids_dict.pop(id)
        except KeyError as e:
            print(f"Not found in IDs dict: {e}")

    with open(f"{ids_path}/ids_dict_{namespace}.json", "w") as f:
        json.dump(current_ids_dict, f, indent=4, ensure_ascii=False)

def upsert_from_documents(load_dir, urls, specify_loader):
    manage_index = ManageIndex(load_dir=load_dir, urls=urls, specify_loader=specify_loader)

    # 資料の重複処理
    docs = manage_index.load_documents()
    source_list = [doc["value"].replace(load_dir+"/", "") for doc in docs]
    duplicate_sorces_list = []
    for item in current_ids_dict.values():
        if item["source"] in source_list:
            duplicate_sorces_list.append(item["source"])
    if duplicate_sorces_list:
        print(f"同じ名前の資料が見つかりました。\n{duplicate_sorces_list}\n")
        user_input = input("上書きする場合は'overwrite'、登録から除外する場合は'exclude'、そのまま登録する場合は'entry'を入力してください。:")
        if user_input == "overwrite":
            print(f"同じ名前の資料を上書きします。\n{duplicate_sorces_list}")
            delete_items(delete_sources_list=duplicate_sorces_list)
        elif user_input == "exclude":
            print(f"同じ名前の資料を登録から除外します。\n{duplicate_sorces_list}")
            manage_index.docs = [doc for doc in manage_index.docs if doc["value"].replace(load_dir+"/", "") not in duplicate_sorces_list]
            print(manage_index.docs)
        elif user_input == "entry":
            print(f"同じ名前の資料を重複させて登録します。\n{duplicate_sorces_list}")
        else:
            print(f"入力が不正です。{user_input}")
            return

    upsert_ids_dict = manage_index.from_documents(namespace, additional_metadata)
    if isinstance(upsert_ids_dict, str):
        print(upsert_ids_dict)
        return
    update_ids_json(upsert_ids_dict=upsert_ids_dict)
    print(f"Upsert {len(upsert_ids_dict)} vector items.")

def upsert_from_items(load_dir):
    manage_index = ManageIndex(load_dir=load_dir)
    upsert_ids_dict = manage_index.from_items(namespace)
    if isinstance(upsert_ids_dict, str):
        print(upsert_ids_dict)
        return
    update_ids_json(upsert_ids_dict=upsert_ids_dict)
    print(f"Upsert {len(upsert_ids_dict)} vector items.")

def delete_items(delete_ids_list=[], delete_sources_list=[]):
    index = pinecone.Index(os.environ["INDEX_NAME"])

    if delete_sources_list:
        for id, value in current_ids_dict.items():
            if value["source"] in delete_sources_list:
                delete_ids_list.append(id)

    index.delete(ids=delete_ids_list, namespace=namespace)
    update_ids_json(delete_ids_list=delete_ids_list)
    print(f"Delete {len(delete_ids_list)} vector items.")

In [ ]:
#@title データ登録 { vertical-output: true }
print("読み込ませるURLまたはファイルを指定してください。")
print("URLのみ読み込ませる場合はCancel uploadをクリックしてください。")
print("フォルダを登録する場合は、圧縮してzipファイルをアップロードしてください。")
print("アイテム修正の場合は、決まったフォーマットのsource_documents.csvをアップロードしてください。\n")

# URLを指定
"""Example:
urls = [
  "https://www.google.com/",
  "https://openai.com/"
]
"""
urls = [

]
print(f"URL list: {urls}\n")

from google.colab import files
import os

os.chdir(load_dir)
uploaded = files.upload()

for fn in uploaded.keys():
    if fn.endswith('.zip'):
        !unzip -o "$fn" -d .
        print(f'Extracted all contents from {fn}')
        !rm "$fn"
        !rm -rf "$load_dir"/__MACOSX
    else:
        print(f'Uploaded {fn}')

os.chdir('/content')


In [ ]:
#@title データ追加

specify_loader = []
# specify_loader = ["PDFMinerLoader"] # 文字化けする場合はこの行のコメントアウトを外して実行

upsert_from_documents(load_dir, urls, specify_loader)
!rm -rf /content/load_files/* # 追加後、アップロードしたファイルは削除される

In [ ]:
#@title アイテムの修正 { vertical-output: true }
upsert_from_items(load_dir)
!rm -rf /content/load_files/* # 追加後、アップロードしたファイルは削除される

In [ ]:
#@title アイテム削除 { vertical-output: true }
print("削除するアイテムのIDのリストを入力してください。")
delete_ids_list = [

]
delete_sources_list=[

]
delete_items(delete_ids_list=delete_ids_list, delete_sources_list=delete_sources_list)

# ids_dict.jsonの全てを削除する場合
# delete_items(delete_ids_list=list(current_ids_dict.keys()))

In [ ]:
#@title Pineconeとids_dictの整合性をとる（Pineconeに合わせる）
index = pinecone.Index(os.environ["INDEX_NAME"])
result = index.query(
    vector=[i for i in range(1536)],
    top_k=10000,
    include_values=False,
    include_metadata=True,
    namespace=namespace
)

# 新しい辞書を作成
new_dict = {}
for item in result["matches"]:
    try:
        new_dict[item["id"]] = item["metadata"]
    except AttributeError:
        new_dict[item["id"]] = {}

update_ids_json(upsert_ids_dict=new_dict)

In [ ]:
#@title Pinecone上の全アイテム削除 { vertical-output: true }
# index = pinecone.Index(os.environ["INDEX_NAME"])
# result = index.query(
#     vector=[i for i in range(1536)],
#     top_k=10000,
#     include_values=True,
#     namespace=namespace
# )

# delete_ids = []
# for vector in result["matches"]:
#     delete_ids.append(vector["id"])

# print(len(delete_ids))
# delete_items(delete_ids_list=delete_ids)